In [1]:
from shutil import disk_usage

# 以下のライブラリを使うので、あらかじめ読み込んでおいてください
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 小数第3位まで表示
%precision 3

# 必要なライブラリのimportを追加
import requests
import zipfile
import io

In [2]:
# 階層型インデックス
# 3列3行のデータを作成し、インデックスとカラムを設定
hier_df = DataFrame(
    np.arange(9).reshape((3,3)),
    index = [
        ['a','a','b'],
        [1,2,2]
    ],
    columns = [
        ['Osaka','Tokyo','Osaka'],
        ['Blue','Red','Red']
    ]
)
hier_df


Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [3]:
# indexに名前をつける
hier_df.index.names = ['key1', 'key2']

# columnsに名前をつける
hier_df.columns.names = ['city', 'color']
hier_df

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

In [4]:
hier_df['Osaka']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

In [5]:
# インデックスを軸にした集計
# print(hier_df.index)
# hier_df.sum(level = 'color', axis = 1)  # colorを軸に集計
hier_df.groupby(level='key2').sum()

# インデックスの 'color' を軸に集計（行方向の groupby）
print(hier_df.groupby(level='color', axis=1).sum())

# インデックスの 'key2' を軸に集計（列方向の場合は transposeしてから）
# print(hier_df.T.groupby(level='key2').sum().T)

# なぜか動かない。。。

color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15


/var/folders/zt/k_5y_7596w56m6k0h09ycxtw0000gn/T/ipykernel_50000/3354836142.py:7: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  print(hier_df.groupby(level='color', axis=1).sum())


In [6]:
# 'b' を key1 レベルでdrop
hier_df.drop('b', level='key1')

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5

In [7]:
# Practice 6-1
hier_df1 = DataFrame(
    np.arange(12).reshape((3,4)),
    index = [['c','d','d'],[1,2,1]],
    columns = [
        ['Kyoto','Nagoya','Hokkaido','Kyoto'],
        ['Yellow','Yellow','Red','Blue']
    ]
)

hier_df1.index.names = ['key1','key2']
hier_df1.columns.names = ['city','color']
hier_df1

city       Kyoto Nagoya Hokkaido Kyoto
color     Yellow Yellow      Red  Blue
key1 key2                             
c    1         0      1        2     3
d    2         4      5        6     7
     1         8      9       10    11

In [8]:
# Kyotoのみを表示
hier_df1['Kyoto']

color      Yellow  Blue
key1 key2              
c    1          0     3
d    2          4     7
     1          8    11

In [9]:
# Practice 6-2
# cityの平均値
# groupbyを使ってcityごとに平均を計算
hier_df1.groupby(axis=1, level='city').mean()

/var/folders/zt/k_5y_7596w56m6k0h09ycxtw0000gn/T/ipykernel_50000/3442171953.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df1.groupby(axis=1, level='city').mean()


city       Hokkaido  Kyoto  Nagoya
key1 key2                         
c    1          2.0    1.5     1.0
d    2          6.0    5.5     5.0
     1         10.0    9.5     9.0

In [10]:
# データの結合
# データ1の準備
data1 = {
    'id': ['100', '101', '102', '103', '104', '106', '108', '110', '111', '113'],
    'city': ['Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo', 'Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo'],
    'birth_year': [1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
    'name': ['Hiroshi', 'Akiko', 'Yuki', 'Satoru', 'Steeve', 'Mituru', 'Aoi', 'Tarou', 'Suguru','Mitsuo']
}
df1 = DataFrame(data1)
df1


,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


In [11]:
# データ2の準備
data2 = {
    'id': ['100', '101', '102', '105', '107'],
    'math': [50, 43, 33, 76, 98],
    'english': [90, 30, 20, 50, 30],
    'sex': ['M', 'F', 'F', 'M', 'M'],
    'index_num': [0, 1, 2, 3, 4]
}

df2 = DataFrame(data2)
df2

,id,math,english,sex,index_num
0,100,50,90,M,0
1,101,43,30,F,1
2,102,33,20,F,2
3,105,76,50,M,3
4,107,98,30,M,4


In [12]:
# 内部結合
print('結合テーブル')
pd.merge(df1, df2, on = 'id')

結合テーブル


,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50,90,M,0
1,101,Osaka,1989,Akiko,43,30,F,1
2,102,Kyoto,1992,Yuki,33,20,F,2


In [13]:
# 全結合
pd.merge(df1, df2, how = 'outer')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990.0,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989.0,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992.0,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997.0,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982.0,Steeve,NaN,NaN,NaN,NaN
5,105,NaN,NaN,NaN,76.0,50.0,M,3.0
6,106,Tokyo,1991.0,Mituru,NaN,NaN,NaN,NaN
7,107,NaN,NaN,NaN,98.0,30.0,M,4.0
8,108,Osaka,1988.0,Aoi,NaN,NaN,NaN,NaN
9,110,Kyoto,1990.0,Tarou,NaN,NaN,NaN,NaN


In [14]:
# indexのよるマージ
pd.merge(df1, df2, left_index = True, right_on = 'index_num')

,id_x,city,birth_year,name,id_y,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,100,50,90,M,0
1,101,Osaka,1989,Akiko,101,43,30,F,1
2,102,Kyoto,1992,Yuki,102,33,20,F,2
3,103,Hokkaido,1997,Satoru,105,76,50,M,3
4,104,Tokyo,1982,Steeve,107,98,30,M,4


In [15]:
# 左外部結合
pd.merge(df1, df2, how = 'left')


,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Tokyo,1991,Mituru,NaN,NaN,NaN,NaN
6,108,Osaka,1988,Aoi,NaN,NaN,NaN,NaN
7,110,Kyoto,1990,Tarou,NaN,NaN,NaN,NaN
8,111,Hokkaido,1995,Suguru,NaN,NaN,NaN,NaN
9,113,Tokyo,1981,Mitsuo,NaN,NaN,NaN,NaN


In [16]:
# 縦結合
# データ3の準備
data3 = {
    'id': ['117', '118', '119', '120', '125'],
    'city': ['Chiba', 'Kanagawa', 'Tokyo', 'Fukuoka', 'Okinawa'],
    'birth_year': [1990, 1989, 1992, 1997, 1982],
    'name': ['Suguru', 'Koichi', 'Satochi', 'Yukie', 'Akari']
}
df3 = DataFrame(data3)
df3


,id,city,birth_year,name
0,117,Chiba,1990,Suguru
1,118,Kanagawa,1989,Koichi
2,119,Tokyo,1992,Satochi
3,120,Fukuoka,1997,Yukie
4,125,Okinawa,1982,Akari


In [17]:
# concat 縦結合
concat_data = pd.concat([df1, df3])
concat_data

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


In [18]:
# データ4の準備
data4 = {
    'id': ['0', '1', '2', '3', '4', '6', '8', '11', '12', '13'],
    'city': ['Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo', 'Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo'],
    'birth_year': [1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
    'name': ['Hiroshi', 'Akiko', 'Yuki', 'Satoru', 'Steeve', 'Mituru', 'Aoi', 'Tarou', 'Suguru', 'Mitsuo']
}
df4 = DataFrame(data4)
df4


,id,city,birth_year,name
0,0,Tokyo,1990,Hiroshi
1,1,Osaka,1989,Akiko
2,2,Kyoto,1992,Yuki
3,3,Hokkaido,1997,Satoru
4,4,Tokyo,1982,Steeve
5,6,Tokyo,1991,Mituru
6,8,Osaka,1988,Aoi
7,11,Kyoto,1990,Tarou
8,12,Hokkaido,1995,Suguru
9,13,Tokyo,1981,Mitsuo


In [19]:
# データ5の準備
data5 = {
    'id': ['0', '1', '3', '6', '8'],
    'math': [20, 30, 50, 70, 90],
    'english': [30, 50, 50, 70, 20],
    'sex': ['M', 'F', 'F', 'M', 'M'],
    'index_num': [0, 1, 2, 3, 4]
}
df5 = DataFrame(data5)
df5


,id,math,english,sex,index_num
0,0,20,30,M,0
1,1,30,50,F,1
2,3,50,50,F,2
3,6,70,70,M,3
4,8,90,20,M,4


In [20]:
# データの準備
data6 = {
    'id': ['70', '80', '90', '120', '150'],
    'city': ['Chiba', 'Kanagawa', 'Tokyo', 'Fukuoka', 'Okinawa'],
    'birth_year': [1980, 1999, 1995, 1994, 1994],
    'name': ['Suguru', 'Koichi', 'Satochi', 'Yukie', 'Akari']
}
df6 = DataFrame(data6)
df6


,id,city,birth_year,name
0,70,Chiba,1980,Suguru
1,80,Kanagawa,1999,Koichi
2,90,Tokyo,1995,Satochi
3,120,Fukuoka,1994,Yukie
4,150,Okinawa,1994,Akari


In [21]:
# 内部結合
pd.merge(df1, df2, on = 'id')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50,90,M,0
1,101,Osaka,1989,Akiko,43,30,F,1
2,102,Kyoto,1992,Yuki,33,20,F,2


In [22]:
# 全結合
pd.merge(df1, df2, how = 'outer')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990.0,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989.0,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992.0,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997.0,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982.0,Steeve,NaN,NaN,NaN,NaN
5,105,NaN,NaN,NaN,76.0,50.0,M,3.0
6,106,Tokyo,1991.0,Mituru,NaN,NaN,NaN,NaN
7,107,NaN,NaN,NaN,98.0,30.0,M,4.0
8,108,Osaka,1988.0,Aoi,NaN,NaN,NaN,NaN
9,110,Kyoto,1990.0,Tarou,NaN,NaN,NaN,NaN


In [23]:
# practice 6-6
# 縦結合
concat_data2 = pd.concat([df4, df6])
concat_data2

,id,city,birth_year,name
0,0,Tokyo,1990,Hiroshi
1,1,Osaka,1989,Akiko
2,2,Kyoto,1992,Yuki
3,3,Hokkaido,1997,Satoru
4,4,Tokyo,1982,Steeve
5,6,Tokyo,1991,Mituru
6,8,Osaka,1988,Aoi
7,11,Kyoto,1990,Tarou
8,12,Hokkaido,1995,Suguru
9,13,Tokyo,1981,Mitsuo


In [24]:
# データの操作と交換
hier_df

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

In [25]:
# ピボット 列を軸にしてデータを変形
hier_df.stack()


/var/folders/zt/k_5y_7596w56m6k0h09ycxtw0000gn/T/ipykernel_50000/660998392.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  hier_df.stack()


city             Osaka  Tokyo
key1 key2 color              
a    1    Blue       0    NaN
          Red        2    1.0
     2    Blue       3    NaN
          Red        5    4.0
b    2    Blue       6    NaN
          Red        8    7.0

In [26]:
hier_df.stack().unstack()

/var/folders/zt/k_5y_7596w56m6k0h09ycxtw0000gn/T/ipykernel_50000/1103922405.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  hier_df.stack().unstack()


city      Osaka     Tokyo     
color      Blue Red  Blue  Red
key1 key2                     
a    1        0   2   NaN  1.0
     2        3   5   NaN  4.0
b    2        6   8   NaN  7.0

In [27]:
# 重複データの除去
# 重複があるデータ
dupli_data = DataFrame({
    'col1': [1, 1, 2, 3, 4, 4, 6, 6, 6],
    'col2': ['a', 'b', 'b', 'b', 'b', 'c', 'c', 'c', 'b']
})

print('・元のデータ')
dupli_data

・元のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,b
5,4,c
6,6,c
7,6,c
8,6,b


In [28]:
# 重複判定
dupli_data.duplicated()  # 重複している行をTrueで返す

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7     True
8    False
dtype: bool

In [29]:
dupli_data.drop_duplicates()

,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,b
5,4,c
6,6,c
8,6,b


In [30]:
# マッピング処理
# 参照データ
city_map = {
    'Tokyo': 'Kanto',
    'Hokkaido': 'Hokkaido',
    'Osaka': 'Kansai',
    'Kyoto': 'Kansai'
}
city_map

{'Tokyo': 'Kanto',
 'Hokkaido': 'Hokkaido',
 'Osaka': 'Kansai',
 'Kyoto': 'Kansai'}

In [31]:
# 参照データ結合
df1['region'] = df1['city'].map(city_map)
df1

,id,city,birth_year,name,region
0,100,Tokyo,1990,Hiroshi,Kanto
1,101,Osaka,1989,Akiko,Kansai
2,102,Kyoto,1992,Yuki,Kansai
3,103,Hokkaido,1997,Satoru,Hokkaido
4,104,Tokyo,1982,Steeve,Kanto
5,106,Tokyo,1991,Mituru,Kanto
6,108,Osaka,1988,Aoi,Kansai
7,110,Kyoto,1990,Tarou,Kansai
8,111,Hokkaido,1995,Suguru,Hokkaido
9,113,Tokyo,1981,Mitsuo,Kanto


In [32]:
# 無名関数とmapの組み合わせ
df1['up_two_num'] = df1['birth_year'].map(lambda x: str(x)[0:3])
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,Kansai,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,Kansai,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [33]:
# ビン分分割
birth_year_bins = [1980, 1985, 1990, 1995, 2000]

birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64, right]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

In [34]:
# 集計結果
birth_year_cut_data.value_counts()

birth_year
(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: count, dtype: int64

In [35]:
# 名前をつける
group_names = ['early1980s', 'late1980s', 'early1990s', 'late1990s']
birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins, labels=group_names)
birth_year_cut_data.value_counts()

birth_year
late1980s     4
early1990s    3
early1980s    2
late1990s     1
Name: count, dtype: int64

In [36]:
student_data_por = pd.read_csv('./data/student-por.csv', sep=';')
student_data_por.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [37]:
# Practice 6-7
student_data_math = pd.read_csv('./data/student-mat.csv', sep=';')
student_data_math['age_x2'] = student_data_math['age'] * 2
student_data_math.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,age_x2
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,36
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,34
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,30
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,30
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,32


In [38]:

absences_bins = [0, 1, 5, 100]
absences_cut = pd.cut(student_data_math['absences'], absences_bins, right=False)
absences_cut.value_counts()


absences
[5, 100)    151
[1, 5)      129
[0, 1)      115
Name: count, dtype: int64

In [39]:
absences_qcut = pd.qcut(student_data_math['absences'], 3)
absences_qcut.value_counts()


absences
(-0.001, 2.0]    183
(6.0, 75.0]      115
(2.0, 6.0]        97
Name: count, dtype: int64

In [40]:
# データの確認
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,Kansai,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,Kansai,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [41]:
#cityの値がいくつあるかを確認
df1.groupby('city').size()

city
Hokkaido    2
Kyoto       2
Osaka       2
Tokyo       4
dtype: int64

In [42]:
# cityを軸にbirth_yearの平均を計算
df1.groupby('city')['birth_year'].mean()

city
Hokkaido    1996.0
Kyoto       1991.0
Osaka       1988.5
Tokyo       1986.0
Name: birth_year, dtype: float64

In [43]:
df1.groupby(['region', 'city'])['birth_year'].mean()

region    city    
Hokkaido  Hokkaido    1996.0
Kansai    Kyoto       1991.0
          Osaka       1988.5
Kanto     Tokyo       1986.0
Name: birth_year, dtype: float64

In [44]:
df1.groupby(['region', 'city'], as_index = False)['birth_year'].mean()

,region,city,birth_year
0,Hokkaido,Hokkaido,1996.0
1,Kansai,Kyoto,1991.0
2,Kansai,Osaka,1988.5
3,Kanto,Tokyo,1986.0


In [45]:
# for文を使ったグループ化
for group, subdf in df1.groupby('region'):
    print('=====================')
    print('Region Name:'.format(group))
    print(subdf)

Region Name:
    id      city  birth_year    name    region up_two_num
3  103  Hokkaido        1997  Satoru  Hokkaido        199
8  111  Hokkaido        1995  Suguru  Hokkaido        199
Region Name:
    id   city  birth_year   name  region up_two_num
1  101  Osaka        1989  Akiko  Kansai        198
2  102  Kyoto        1992   Yuki  Kansai        199
6  108  Osaka        1988    Aoi  Kansai        198
7  110  Kyoto        1990  Tarou  Kansai        199
Region Name:
    id   city  birth_year     name region up_two_num
0  100  Tokyo        1990  Hiroshi  Kanto        199
4  104  Tokyo        1982   Steeve  Kanto        198
5  106  Tokyo        1991   Mituru  Kanto        199
9  113  Tokyo        1981   Mitsuo  Kanto        198


In [46]:
# Practice 6-10
student_data_math.groupby('school')['G1'].mean()

school
GP    10.939828
MS    10.673913
Name: G1, dtype: float64

In [47]:
# Practice 6-11
student_data_math.groupby(['school', 'sex'])[['G1', 'G2', 'G3']].mean()

G1         G2         G3
school sex                                 
GP     F    10.579235  10.398907   9.972678
       M    11.337349  11.204819  11.060241
MS     F    10.920000  10.320000   9.920000
       M    10.380952  10.047619   9.761905

In [48]:
# Practice 6-1っ2
functions = ['max', 'min']
student_data_math2 = student_data_math.groupby(['school','sex'])
student_data_math2[['G1', 'G2', 'G3']].agg(functions)

G1      G2      G3    
           max min max min max min
school sex                        
GP     F    18   4  18   0  19   0
       M    19   3  19   0  20   0
MS     F    19   6  18   5  19   0
       M    15   6  16   5  16   0

In [50]:
# データの準備
import numpy as np
from numpy import nan as NA
import pandas as pd

df = pd.DataFrame(np.random.rand(10, 4))

# NAにする
df.iloc[1,0] = NA
df.iloc[2:3,2] = NA
df.iloc[5:,3] = NA

df

,0,1,2,3
0,0.540286,0.587269,0.154972,0.159394
1,NaN,0.715093,0.896905,0.181452
2,0.274983,0.573667,NaN,0.902219
3,0.281157,0.700837,0.966771,0.665780
4,0.192647,0.593624,0.145138,0.710150
5,0.554963,0.395562,0.919780,NaN
6,0.876872,0.781947,0.335437,NaN
7,0.209331,0.742724,0.498601,NaN
8,0.845410,0.825532,0.214020,NaN
9,0.091066,0.448000,0.822126,NaN


In [52]:
df.dropna()

,0,1,2,3
0,0.540286,0.587269,0.154972,0.159394
3,0.281157,0.700837,0.966771,0.665780
4,0.192647,0.593624,0.145138,0.710150


In [53]:
# ペアワイズ削除
df[[0,1]].dropna()  # 列0と列1のペアワイズ削除

,0,1
0,0.540286,0.587269
2,0.274983,0.573667
3,0.281157,0.700837
4,0.192647,0.593624
5,0.554963,0.395562
6,0.876872,0.781947
7,0.209331,0.742724
8,0.845410,0.825532
9,0.091066,0.448000


In [54]:
# fillnaを使った欠損値の補完
df.fillna(0)  # 0で補完

,0,1,2,3
0,0.540286,0.587269,0.154972,0.159394
1,0.000000,0.715093,0.896905,0.181452
2,0.274983,0.573667,0.000000,0.902219
3,0.281157,0.700837,0.966771,0.665780
4,0.192647,0.593624,0.145138,0.710150
5,0.554963,0.395562,0.919780,0.000000
6,0.876872,0.781947,0.335437,0.000000
7,0.209331,0.742724,0.498601,0.000000
8,0.845410,0.825532,0.214020,0.000000
9,0.091066,0.448000,0.822126,0.000000


In [56]:
# 前の行で補完
df.ffill() # 前の行の値で補完

,0,1,2,3
0,0.540286,0.587269,0.154972,0.159394
1,0.540286,0.715093,0.896905,0.181452
2,0.274983,0.573667,0.896905,0.902219
3,0.281157,0.700837,0.966771,0.665780
4,0.192647,0.593624,0.145138,0.710150
5,0.554963,0.395562,0.919780,0.710150
6,0.876872,0.781947,0.335437,0.710150
7,0.209331,0.742724,0.498601,0.710150
8,0.845410,0.825532,0.214020,0.710150
9,0.091066,0.448000,0.822126,0.710150


In [57]:
# 各カラムの平均値
df.mean()  # 各カラムの平均値

0    0.429635
1    0.636426
2    0.550417
3    0.523799
dtype: float64

In [58]:
# 各カラムの平均値で補完
df.fillna(df.mean())  # 各カラムの平均値で補完

,0,1,2,3
0,0.540286,0.587269,0.154972,0.159394
1,0.429635,0.715093,0.896905,0.181452
2,0.274983,0.573667,0.550417,0.902219
3,0.281157,0.700837,0.966771,0.665780
4,0.192647,0.593624,0.145138,0.710150
5,0.554963,0.395562,0.919780,0.523799
6,0.876872,0.781947,0.335437,0.523799
7,0.209331,0.742724,0.498601,0.523799
8,0.845410,0.825532,0.214020,0.523799
9,0.091066,0.448000,0.822126,0.523799


In [59]:
# データの準備
import numpy as np
from numpy import nan as NA
import pandas as pd

df2 = pd.DataFrame(np.random.rand(15,6))

# NAにする
df2.iloc[2,0] = NA
df2.iloc[5:9,2] = NA
df2.iloc[7:10,3] = NA
df2.iloc[10,5] = NA

df2

,0,1,2,3,4,5
0,0.899696,0.555858,0.034604,0.498696,0.926985,0.298636
1,0.579955,0.963951,0.942478,0.974230,0.256888,0.633529
2,NaN,0.012631,0.621282,0.807623,0.094854,0.407945
3,0.851046,0.298020,0.757961,0.861038,0.218766,0.269335
4,0.616498,0.600713,0.095518,0.849518,0.709381,0.351556
5,0.801245,0.745209,NaN,0.768584,0.700482,0.221479
6,0.661768,0.193255,NaN,0.635691,0.951165,0.468482
7,0.565521,0.994895,NaN,NaN,0.335984,0.278484
8,0.218214,0.041085,NaN,NaN,0.081528,0.894961
9,0.209538,0.636200,0.923403,NaN,0.405871,0.666959


In [60]:
# 欠損値を含む行を削除
df2.dropna()  # 欠損値を含む行を削除

,0,1,2,3,4,5
0,0.899696,0.555858,0.034604,0.498696,0.926985,0.298636
1,0.579955,0.963951,0.942478,0.974230,0.256888,0.633529
3,0.851046,0.298020,0.757961,0.861038,0.218766,0.269335
4,0.616498,0.600713,0.095518,0.849518,0.709381,0.351556
11,0.923541,0.236429,0.766548,0.889037,0.773181,0.441865
12,0.675201,0.348630,0.083940,0.711020,0.678261,0.190825
13,0.486639,0.533158,0.542734,0.086807,0.818262,0.181914
14,0.546635,0.168453,0.852662,0.704780,0.594226,0.387749


In [61]:
# 0で欠損値を埋める
df.fillna(0)  # 0で欠損値を埋める

,0,1,2,3
0,0.540286,0.587269,0.154972,0.159394
1,0.000000,0.715093,0.896905,0.181452
2,0.274983,0.573667,0.000000,0.902219
3,0.281157,0.700837,0.966771,0.665780
4,0.192647,0.593624,0.145138,0.710150
5,0.554963,0.395562,0.919780,0.000000
6,0.876872,0.781947,0.335437,0.000000
7,0.209331,0.742724,0.498601,0.000000
8,0.845410,0.825532,0.214020,0.000000
9,0.091066,0.448000,0.822126,0.000000


In [62]:
# 平均値で埋める

df2.fillna(df2.mean())  # 各カラムの平均値で欠損値を埋める

,0,1,2,3,4,5
0,0.899696,0.555858,0.034604,0.498696,0.926985,0.298636
1,0.579955,0.963951,0.942478,0.974230,0.256888,0.633529
2,0.612973,0.012631,0.621282,0.807623,0.094854,0.407945
3,0.851046,0.298020,0.757961,0.861038,0.218766,0.269335
4,0.616498,0.600713,0.095518,0.849518,0.709381,0.351556
5,0.801245,0.745209,0.543995,0.768584,0.700482,0.221479
6,0.661768,0.193255,0.543995,0.635691,0.951165,0.468482
7,0.565521,0.994895,0.543995,0.656360,0.335984,0.278484
8,0.218214,0.041085,0.543995,0.656360,0.081528,0.894961
9,0.209538,0.636200,0.923403,0.656360,0.405871,0.666959


In [66]:
# chapter6-4 時系列データの取り扱い
import pandas_datareader.data as pdr

start_date = '2001/1/2'
end_date = '2016/12/30'

fx_jpusdata = pdr.DataReader('DEXJPUS', 'fred', start_date, end_date)
fx_jpusdata.head()


,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


In [70]:
# 特定の年月のデータを参照する
fx_jpusdata.loc['2016-04']

,DEXJPUS
DATE,
2016-04-01,112.06
2016-04-04,111.18
2016-04-05,110.26
2016-04-06,109.63
2016-04-07,107.98
2016-04-08,108.36
2016-04-11,107.96
2016-04-12,108.54
2016-04-13,109.21


In [74]:
fx_jpusdata.resample('ME').last().head()

,DEXJPUS
DATE,
2001-01-31,116.39
2001-02-28,117.28
2001-03-31,125.54
2001-04-30,123.57
2001-05-31,118.88


In [75]:
# 欠損値がある場合の操作
fx_jpusdata.resample('D').last().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,NaN


In [76]:
fx_jpusdata.resample('D').ffill().head()  # 前の値で補完

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,116.19


In [77]:
# データをずらして比率を計算する
fx_jpusdata.shift(1).head()  # 1日ずらす

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,114.73
2001-01-04,114.26
2001-01-05,115.47
2001-01-08,116.19


In [79]:
# 年ごとの平均値
fx_jpusdata.resample('YE').mean()

,DEXJPUS
DATE,
2001-12-31,121.568040
2002-12-31,125.220438
2003-12-31,115.938685
2004-12-31,108.150830
2005-12-31,110.106932
2006-12-31,116.312072
2007-12-31,117.762323
2008-12-31,103.390635
2009-12-31,93.682659


In [80]:
# 移動平均
fx_jpusdata.rolling(3).mean().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,114.820000
2001-01-05,115.306667
2001-01-08,115.876667


In [81]:
# 標準偏差
fx_jpusdata.rolling(3).std().head()  # 3日間の移動標準偏差

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,0.610000
2001-01-05,0.975312
2001-01-08,0.368963


In [82]:
# Practice 6-17 nanを削除
fx_jpusdata.rolling(20).mean().dropna()  # 20日間の移動平均を計算し、nanを削除

,DEXJPUS
DATE,
2001-02-12,116.6910
2001-02-13,116.6920
2001-02-14,116.6070
2001-02-15,116.5015
2001-02-16,116.4130
...,...
2016-11-08,104.1600
2016-11-09,104.1780
2016-11-10,104.3250


In [88]:
# 総合問題6-1
# # ageとsexでG1の平均点を出す（縦軸: age, 横軸: sex）
student_data_math.pivot_table(values='G1', index='age', columns='sex', aggfunc='mean')

sex,F,M
age,,
15,10.052632,12.250000
16,10.203704,11.740000
17,11.103448,10.600000
18,10.883721,10.538462
19,10.642857,9.700000
20,15.000000,13.000000
21,NaN,10.000000
22,NaN,6.000000


In [89]:
# NaNを含むデータを削除
student_data_math.pivot_table(values='G1', index='age', columns='sex', aggfunc='mean').dropna()

sex,F,M
age,,
15,10.052632,12.250000
16,10.203704,11.740000
17,11.103448,10.600000
18,10.883721,10.538462
19,10.642857,9.700000
20,15.000000,13.000000
